In [1]:
#importing necessary libraries
import csv
import pandas as pd
import requests
import json
import datetime

In [2]:
def get_pushshift_data(after, before, sub):
    
    url = 'https://api.pushshift.io/reddit/search/submission/?&after=' + str(after) + '&before=' + str(before) + '&subreddit=' + str(sub)
    r = requests.get(url)
    data = json.loads(r.text, strict=False)
    return data['data']

In [3]:
def collect_sub_data(subm):
    
    subData = list()  # list to store data points
    title = subm['title']
    url = subm['url']
    try:
        flair = subm['link_flair_text']
    except KeyError:
        flair = "NaN"
    try:
        body = subm['selftext']
    except KeyError:
        body = ''
    author = subm['author']
    sub_id = subm['id']
    score = subm['score']
    created = datetime.datetime.fromtimestamp(subm['created_utc'])  # 1520561700.0
    numComms = subm['num_comments']
    permalink = subm['permalink']

    subData.append((sub_id, title, body, url, author, score, created, numComms, permalink, flair))
    subStats[sub_id] = subData


In [4]:
sub = 'depression'
# before and after dates
before = "1601596800"  
after = "1601510400"  

subCount = 0
subStats = {}

In [5]:
data = get_pushshift_data(after, before, sub)  # Will run until all posts have been gathered from the 'after' date up until 'before' date
while len(data) > 0:
    for submission in data:
        collect_sub_data(submission)
        subCount += 1
    # Calls getPushshiftData() with the created date of the last submission
    after = data[-1]['created_utc']
    data = get_pushshift_data(after, before, sub)

print(str(len(subStats)) + " submissions have added to list")
print("1st entry is:")
print(list(subStats.values())[0][0][1] + " created: " + str(list(subStats.values())[0][0][5]))
print("Last entry is:")
print(list(subStats.values())[-1][0][1] + " created: " + str(list(subStats.values())[-1][0][5]))


418 submissions have added to list
1st entry is:
An Erratic Schedule created: 1
Last entry is:
I can’t take it anymore created: 1


In [6]:
def updateSubs_file():
    
    upload_count = 0
    location = "C:/Users/SurbhiLaptop/RedditData/"
    print("input filename of submission file, please add .csv")
    filename = input()
    file = location + filename
    with open(file, 'w', newline='', encoding='utf-8') as file:
        a = csv.writer(file, delimiter=',')
        headers = ["Post ID", "Title", "Body", "Url", "Author", "Score", "Publish Date", "Total No. of Comments", "Permalink",
                   "Flair"]
        a.writerow(headers)
        for sub in subStats:
            a.writerow(subStats[sub][0])
            upload_count += 1

        print(str(upload_count) + " submissions have been uploaded")
        
updateSubs_file()        

input filename of submission file, please add .csv
Dataset.csv
418 submissions have been uploaded
